In [ ]:
import pandas as pd
import numpy as np
import holidays
from datetime import datetime, timedelta
import random

# --- CONFIGURAÇÕES GERAIS ---
NUM_PRODUTOS = 10  # Vamos simular 10 produtos diferentes
DATA_INICIO = '2023-01-01'
DATA_FIM = '2024-12-31' # 2 anos de dados
ESTOQUE_INICIAL_MAX = 200
ESTOQUE_MINIMO_REPOSICAO = 20

# Configuração dos Feriados BR
br_holidays = holidays.BR()

# Lista de produtos fictícios
produtos = [
    {"id": 101, "nome": "Smartphone Galaxy", "preco_base": 1200.00},
    {"id": 102, "nome": "Notebook Dell", "preco_base": 3500.00},
    {"id": 103, "nome": "Fone Bluetooth", "preco_base": 150.00},
    {"id": 104, "nome": "Monitor 24pol", "preco_base": 800.00},
    {"id": 105, "nome": "Teclado Mecanico", "preco_base": 250.00},
    {"id": 106, "nome": "Mouse Gamer", "preco_base": 120.00},
    {"id": 107, "nome": "Cadeira Ergo", "preco_base": 900.00},
    {"id": 108, "nome": "Webcam HD", "preco_base": 200.00},
    {"id": 109, "nome": "HD Externo 1TB", "preco_base": 300.00},
    {"id": 110, "nome": "Smartwatch", "preco_base": 600.00},
]

data_list = []

# --- GERANDO DADOS ---
dates = pd.date_range(start=DATA_INICIO, end=DATA_FIM)

for prod in produtos:
    estoque_atual = random.randint(50, ESTOQUE_INICIAL_MAX)

    for dia in dates:
        # 1. Definição de Preço (Variação de +/- 10%)
        variacao = random.uniform(0.9, 1.1)
        preco_dia = round(prod["preco_base"] * variacao, 2)

        # 2. Definição de Promoção (10% de chance de ter promoção)
        flag_promocao = 1 if random.random() < 0.10 else 0
        if flag_promocao:
            preco_dia = round(preco_dia * 0.85, 2) # 15% de desconto na promoção

        # 3. Lógica de Demanda Base (Sazonalidade simples)
        demanda_base = random.randint(1, 10) # Venda média normal
        fator_multiplicador = 1.0

        # Se tiver promoção, vende mais
        if flag_promocao:
            fator_multiplicador *= 2.5

        # --- LÓGICA DE FERIADOS (A parte complexa) ---
        is_feriado = dia in br_holidays
        dia_da_semana = dia.weekday() # 0=Seg, 4=Sex, 5=Sab, 6=Dom

        # Verifica se é véspera de feriado prolongado (feriado na Seg ou Sex)
        dia_seguinte = dia + timedelta(days=1)
        dia_pos_seguinte = dia + timedelta(days=2)

        # Lógica: Feriado Prolongado (Cai na Seg ou Sex)
        if is_feriado and (dia_da_semana == 0 or dia_da_semana == 4):
            # No dia do feriado prolongado, vendas caem
            fator_multiplicador *= 0.2
        elif (dia_seguinte in br_holidays and dia_seguinte.weekday() in [0, 4]) or \
             (dia_pos_seguinte in br_holidays and dia_pos_seguinte.weekday() == 0):
            # Nos dias que antecedem o feriado prolongado, vendas sobem
            fator_multiplicador *= 1.8

        # Lógica: Feriado Meio de Semana (Ter, Qua, Qui)
        elif is_feriado and dia_da_semana in [1, 2, 3]:
            # Vendas sobem um pouco
            fator_multiplicador *= 1.3

        # Cálculo final da demanda (Vendas)
        vendas = int(demanda_base * fator_multiplicador)

        # Ajuste de estoque (não pode vender mais do que tem)
        if vendas > estoque_atual:
            vendas = estoque_atual

        estoque_atual -= vendas

        # Lógica de Reposição de Estoque
        if estoque_atual < ESTOQUE_MINIMO_REPOSICAO:
            reposicao = random.randint(50, 100)
            estoque_atual += reposicao

        # Adiciona ao dataset
        data_list.append({
            "ID_PRODUTO": prod["id"],
            "DATA_EVENTO": dia.strftime('%Y-%m-%d'),
            "PRECO": preco_dia,
            "FLAG_PROMOCAO": flag_promocao,
            "FERIADO": 1 if is_feriado else 0,
            "QUANTIDADE_ESTOQUE": estoque_atual,
            "QUANTIDADE_VENDIDA": vendas # Esta será sua variável Target (Alvo) no SageMaker
        })

# Criando DataFrame e Salvando
df = pd.DataFrame(data_list)
print(f"Dataset gerado com {len(df)} linhas.")
df.to_csv("dataset_estoque_inteligente.csv", index=False)
print("Arquivo 'dataset_estoque_inteligente.csv' salvo com sucesso!")

# Exibe as primeiras linhas para conferência
print(df.head())

Dataset gerado com 7310 linhas.
Arquivo 'dataset_estoque_inteligente.csv' salvo com sucesso!
   ID_PRODUTO DATA_EVENTO    PRECO  FLAG_PROMOCAO  FERIADO  \
0         101  2023-01-01  1127.59              0        1   
1         101  2023-01-02  1247.27              0        0   
2         101  2023-01-03  1186.59              0        0   
3         101  2023-01-04  1115.45              0        0   
4         101  2023-01-05  1136.34              0        0   

   QUANTIDADE_ESTOQUE  QUANTIDADE_VENDIDA  
0                 126                   7  
1                 124                   2  
2                 116                   8  
3                 106                  10  
4                 100                   6  
